In [15]:
# load generated questions
import os
import json
import numpy as np
from collections import Counter, defaultdict

topics = defaultdict(list)
qas = defaultdict(list)
non_unique_questions = defaultdict(list)

squash_path = '/home/svakule/squash-generation/squash/temp/trec_cast19_paragraphs'

generated_questions = {}
for p_id in os.listdir(squash_path):
    try:
        with open(squash_path+'/%s/generated_questions.json' % p_id) as f:
            paragraphs = json.load(f)['data'][0]['paragraphs']
            generated_questions[p_id] = []
            for i, p in enumerate(paragraphs):
                for qa in p['qas']:
                    q = qa['question']#.lower()
                    generated_questions[p_id].append(q)
    except:
        continue
                    
print(len(generated_questions), 'passages with generated questions')

6475 passages with generated questions


In [13]:
# filter only questions that have enough support ie relevant passages
# same as in TREC
min_passages_per_question = 3

doc_id = 0
documents = []
all_qs = []
for p_id, qs in generated_questions.items():
    # index questions with sufficient support (non-unique within topic)
    if len(qs) < min_passages_per_question:
        continue
    for q in qs:
        # skip duplicate questions
        if q in all_qs:
            continue
        documents.append({'id': doc_id,
                          'contents': q})
        all_qs.append(q)
        doc_id += 1
print(len(documents), 'unique generated questions')

88658 unique generated questions


# Index questions

In [16]:
import os
import json

anserini_folder = "/home/svakule/markers_bert/Anserini/"
json_files_path = "../results/questions_index/collection"
path_index = "/home/svakule/markers_bert/Anserini/indexes/squash_cast19"

if not os.path.isdir(json_files_path):
    os.makedirs(json_files_path)
for i, doc in enumerate(documents):
    with open(json_files_path+'/docs{:02d}.json'.format(i), 'w', encoding='utf-8', ) as f:
        f.write(json.dumps(doc) + '\n')        

# Run index java command
os.system("sh {}target/appassembler/bin/IndexCollection -collection JsonCollection"   \
            " -generator DefaultLuceneDocumentGenerator -threads 9 -input {}" \
            " -index {} -storePositions -storeDocvectors -storeRaw". \
            format(anserini_folder, json_files_path, path_index))

0

In [20]:
# test search 
import json
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher(path_index)
query_str = 'hubble space telescope'
num_candidates_samples = 3

hits = searcher.search(query_str, k=num_candidates_samples)
# sampled_initial = [ hit.raw for hit in searcher.search(query_str, k=num_candidates_samples)]
# print(sampled_initial)
# Print the first 3 hits:
for i in range(0, num_candidates_samples):
    print(f'{hits[i].score:.5f}', json.loads(hits[i].raw)['contents'])

7.50800 What is the abbreviation for the space telescope?
6.75080 What did the A3 have to do with the space telescope?
6.53120 What is the name of the space telescope that is scheduled to launch in September of 2018?
